In [ ]:
%pylab inline
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
import matplotlib.patches as mpatches
import cartopy
from datetime import timedelta as delta
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import xarray as xr
from os import environ
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.ticker as mticker

In [ ]:
countrylats = [-60, -18, -3, 1.5, 7.3, 8.1, 11.0, 12.8, 13.7, 14.6, 32.5]
countrynames = ['Chile', 'Peru', 'Ecuador', 'Colombia', 'Panama', 'Costa Rica', 'Nicaragua', 'El Salvador', 'Guatemala', 'Mexico']
galapagos_extent = [-91.8, -89, -1.4, 0.7]

In [ ]:
environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

def load_data(exp):
    T = {}
    pfile = xr.open_dataset('/Volumes/oceanparcels/output_data/data_Erik/GalapagosBasinPaper/galapagosparticles_%s.nc' %exp, decode_cf=True)
    T['lon'] = np.ma.filled(pfile.variables['lon'], np.nan)
    T['lat'] = np.ma.filled(pfile.variables['lat'], np.nan)
    T['time'] = np.ma.filled(pfile.variables['time'], np.nan)
    T['age'] = np.ma.filled(pfile.variables['age'], np.nan)
    if 'fishinghours' in pfile.variables:
        T['fishinghours'] = np.ma.filled(pfile.variables['fishinghours'], np.nan)

    if 'bwd' not in exp:
        T['visitedgalapagos'] = np.ma.filled(pfile.variables['visitedgalapagos'], np.nan)
    else:
        T['visitedgalapagos'] = np.ones(T['lat'].shape)
    pfile.close()

    if 'bwd' not in exp:
        T['visitedtime'] = np.nan*np.zeros(T['visitedgalapagos'].shape[0])
        for p in range(T['visitedgalapagos'].shape[0]):
            if np.any(T['visitedgalapagos'][p, :] == 1):
                I = np.where(T['visitedgalapagos'][p, :] == 1)[0][0]
                T['visitedtime'][p] = T['age'][p, I]/86400.
                T['visitedgalapagos'][p, :] = 1
        T['nvisited'] = len(np.where(T['visitedgalapagos'][:, 0]==1)[0])

    if 'fishinghours' in pfile.variables:
        uniquevar = 'fishinghours'
    else:
        uniquevar = 'lat'    
    T['nsites'] = np.unique(T[uniquevar][:, 0]).shape[0]
    T['startsite'] = np.nan*np.zeros(T[uniquevar].shape[0])
    T['startcountry'] = np.nan*np.zeros(T[uniquevar].shape[0])
    for s in range(T['nsites']):
        I = np.where(T[uniquevar][:, 0] == T[uniquevar][s, 0])
        T['startsite'][I] = s
        if 'americas'in exp:
            C = np.where(T[uniquevar][s, 0] > countrylats)[0]
            T['startcountry'][I] = C[-1]

    return T

In [ ]:
exps = ['bwd_v2', 'bwd_wstokes_v2']
titles = {'bwd_v2': 'Currents only from Galapagos', 'bwd_wstokes_v2': 'Currents and waves from Galapagos'}
D = {}
for exp in exps:
    D[exp] = load_data(exp) 

In [ ]:
from matplotlib import colors
pylab.rcParams['figure.figsize'] = (8, 7)

projection = cartopy.crs.SouthPolarStereo(central_longitude=-90)
fig, ax = plt.subplots(1, len(D), subplot_kw={'projection': projection})

if len(D) == 1:
    ax = [ax]

for i, exp in enumerate(exps):
    T = D[exp]
    ax[i].coastlines()
    ax[i].add_feature(cartopy.feature.LAND)
    gl = ax[i].gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=False,
                         linewidth=1, color='gray', alpha=0.5, linestyle='-')
    gl.xlines = False
    gl.ylocator = mticker.FixedLocator([-60, -30, 0, 30])

    
    if 'bwd'in exp:
        bins = [np.arange(-1,361, 1), np.arange(-90, 45, 1)]
        recfc = 'red'

        ax[i].add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                height=galapagos_extent[3]-galapagos_extent[2], facecolor=recfc, alpha=1, linewidth=1,
                                edgecolor='k', transform=cartopy.crs.Geodetic()))

    lon = np.array([lon % 360 for lon in T['lon']])
    H, xe, ye = np.histogram2d(lon[~np.isnan(lon)], T['lat'][~np.isnan(T['lat'])], bins=bins)
    H[0,: ] = H[-1, :]
    xb = (xe[1:] + xe[:-1])/2
    yb = (ye[1:] + ye[:-1])/2

    levels = [10**x for x in range(0, 9)]
    co = ax[i].contourf(xb, yb, H.T, levels=levels, transform=cartopy.crs.PlateCarree(), norm=colors.LogNorm())
    ax[i].set_title(titles[exp])
    
    print(ax[i].get_xlim(), ax[i].get_ylim())
    ax[i].set_xlim([-14678464.887996294, 8843973.281354474])
    ax[i].set_ylim([-12555957.896613397, 22597393.47540219])

cbaxes = fig.add_axes([0.1, 0.05, 0.8, 0.03]) 
cb = plt.colorbar(co, orientation="horizontal", cax = cbaxes)

plt.tight_layout()
savefig('histogram_bwd.pdf')

In [ ]:
exps = ['americas_v2_5yr', 'americas_wstokes_v2_5yr']
titles = {'americas_v2_5yr': 'Currents from Americas', 'americas_wstokes_v2_5yr': 'Currents+waves from Americas'}
exp_colors = {'americas_v2_5yr': 'C8', 'americas_wstokes_v2_5yr': 'C9'}

D = {}
for exp in exps:
    D[exp] = load_data(exp)

In [ ]:
from matplotlib import colors
pylab.rcParams['figure.figsize'] = (8, 10)

projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(len(D), 1, subplot_kw={'projection': projection})
if len(D) == 1:
    ax = [ax]

for i, exp in enumerate(exps):
    T = D[exp]
    ax[i].coastlines()
    ax[i].add_feature(cartopy.feature.LAND)

    if 'americas'in exp:
        bins = [np.arange(-260, -60, 1), np.arange(-60, 45, 1)]
        recfc = 'none'
    elif exp == 'china':
        bins = [np.arange(90, 200, 1), np.arange(-20, 50, 1)]
        recfc = 'none'

    ax[i].add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                    height=galapagos_extent[3]-galapagos_extent[2], facecolor=recfc, alpha=1, linewidth=2,
                                    edgecolor='k', transform=cartopy.crs.Geodetic()))

    H, xe, ye = np.histogram2d(T['lon'][~np.isnan(T['lon'])], T['lat'][~np.isnan(T['lat'])], bins=bins)
    xb = (xe[1:] + xe[:-1])/2
    yb = (ye[1:] + ye[:-1])/2

    levels = [10**x for x in range(0, 9)]
    co = ax[i].contourf(xb, yb, H.T, levels=levels, transform=cartopy.crs.PlateCarree(), norm=colors.LogNorm())
    ax[i].set_extent([bins[0][0], bins[0][-1], bins[1][0], bins[1][-1]], crs=cartopy.crs.PlateCarree())
    ax[i].set_title(titles[exp])

cbaxes = fig.add_axes([0.1, 0.05, 0.8, 0.03]) 
cb = plt.colorbar(co, orientation="horizontal", cax = cbaxes)

# plt.tight_layout()
savefig('histogram_americas.pdf')

In [ ]:
from matplotlib import colors
pylab.rcParams['figure.figsize'] = (8, 10)

projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(len(D), 1, subplot_kw={'projection': projection})
if len(D) == 1:
    ax = [ax]

for i, exp in enumerate(exps):
    T = D[exp]
    ax[i].coastlines()
    ax[i].add_feature(cartopy.feature.LAND)

    if 'americas'in exp:
        bins = [np.arange(-260, -60, 1), np.arange(-60, 45, 1)]
        recfc = 'none'
    elif exp == 'china':
        bins = [np.arange(90, 200, 1), np.arange(-20, 50, 1)]
        recfc = 'none'

    ax[i].add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                    height=galapagos_extent[3]-galapagos_extent[2], facecolor=recfc, alpha=1, linewidth=2,
                                    edgecolor='k', transform=cartopy.crs.Geodetic()))

    nT = np.shape(T['lon'])[0]
    I = np.random.choice(range(nT), int(nT/2))
    lons = T['lon'][I, :]
    lats = T['lat'][I, :]
    H, xe, ye = np.histogram2d(lons[~np.isnan(lons)], lats[~np.isnan(lats)], bins=bins)
    xb = (xe[1:] + xe[:-1])/2
    yb = (ye[1:] + ye[:-1])/2

    levels = [(10**x) for x in range(0, 9)]
    co = ax[i].contourf(xb, yb, H.T, levels=levels, transform=cartopy.crs.PlateCarree(), norm=colors.LogNorm())
    ax[i].set_extent([bins[0][0], bins[0][-1], bins[1][0], bins[1][-1]], crs=cartopy.crs.PlateCarree())
    ax[i].set_title(titles[exp])

cbaxes = fig.add_axes([0.1, 0.05, 0.8, 0.03]) 
cb = plt.colorbar(co, orientation="horizontal", cax = cbaxes)
# plt.tight_layout()
savefig('histogram_americas_comparehalfparticles.pdf')

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 4)

years = range(2000, 2011)

fig, ax = plt.subplots(1, len(D))
if len(D) == 1:
    ax = [ax]

for i, exp in enumerate(exps):
    T = D[exp]

    yrs = T['time'][:, 0].astype('datetime64[Y]').astype('int')+1970
    countriestoplot = [1, 2, 3]
    fracperyear = np.nan*np.zeros((len(countriestoplot), len(years)))
    for c, country in enumerate(countriestoplot):
        for y in range(len(years)):
            I = np.where(np.logical_and(yrs == years[y], T['startcountry'] == country))[0]
            J = np.where(np.isfinite(T['visitedtime'][I]))[0]
            if len(I) > 0:
                fracperyear[c, y] = 100.0*len(J)/len(I)

    lineObjects = ax[i].plot([y+0.5 for y in years], fracperyear.T, 'o-')
    ax[i].legend(iter(lineObjects), [countrynames[c] for c in countriestoplot])
    ax[i].set_xticks(years)
    ax[i].set_xlabel('Year')
    ax[i].set_ylabel('Fraction of particles that pass through Galapagos [%]')
    ax[i].set_title(titles[exp])

    yl = [0, 15]
    for elnino in [2003, 2005, 2007]:
        ax[i].add_collection(PatchCollection([Rectangle((elnino-1./12, 0), 3./12, yl[1])], 
                             facecolor='r', alpha=0.2, edgecolor=None))
        ax[i].text(elnino+0.5/12., yl[1]-0.5, 'EL', horizontalalignment='center')

    for lanina in [2000, 2001, 2008, 2009, 2010]:
        ax[i].add_collection(PatchCollection([Rectangle((lanina-1./12, 0), 3./12, yl[1])], 
                             facecolor='b', alpha=0.2, edgecolor=None))
        ax[i].text(lanina+0.5/12., yl[1]-0.5, 'LN', horizontalalignment='center')
    ax[i].set_ylim(yl)

plt.tight_layout()
savefig('timeseries_ENSOrelation_Americas.pdf')

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 6)

for i, exp in enumerate(exps):
    T = D[exp]
    if np.any(np.isfinite(T['visitedtime'])):
        plt.hist(T['visitedtime'][np.isfinite(T['visitedtime'])]/365., np.arange(0, 5.01, 1./12), 
                 color=exp_colors[exp], alpha=0.5, label=titles[exp])
plt.gca().set_xlim([0, 5])
plt.gca().set_xticks(np.arange(0, 5.1, 0.5))
plt.legend()
plt.xlabel('Time to first pass through Galapagos [years]')
plt.ylabel('Number of particles')

plt.tight_layout()
savefig('timehistogram_Americas.pdf')

In [ ]:
pylab.rcParams['figure.figsize'] = (6, 4)
for i, exp in enumerate(exps):
    T = D[exp]

    pct_visitgalapagos = np.zeros(T['nsites'])
    startlat = np.zeros(T['nsites'])
    for s in range(T['nsites']):
        I = np.where(T['startsite']==s)[0]
        nvisit = len(np.where(T['visitedgalapagos'][I,0]==1)[0])
        pct_visitgalapagos[s] = 100.*nvisit/len(I)
        startlat[s] = T['lat'][I[0], 0]
    linewidth = 2 if i == 0 else 0.8
    plt.plot(startlat, pct_visitgalapagos,'.-', color=exp_colors[exp], label=titles[exp], linewidth=linewidth)

plt.xlabel('Starting latitude [degrees N]')
plt.ylabel('Fraction of particles passing through Galapagos [%]')
plt.legend()
ylim = [0, 30]
xlim = [-25, 25]
for s in range(1, len(countrylats)):
    plt.plot([countrylats[s], countrylats[s]], ylim, 'k:')
    xmin = max(xlim[0], countrylats[s-1])
    xmax = min(xlim[1], countrylats[s])
    plt.text((xmin+xmax)/2, ylim[1]-8, countrynames[s-1], horizontalalignment='center', rotation=90)
plt.xlim(xlim)
plt.ylim(ylim)

plt.tight_layout()
# plt.show()
savefig('lathistogram_Americas.pdf')

In [ ]:
pylab.rcParams['figure.figsize'] = (6, 4)
for i, exp in enumerate(exps):
    T = D[exp]

    pct_visitgalapagos = np.zeros(T['nsites'])
    startlat = np.zeros(T['nsites'])
    for s in range(T['nsites']):
        I = np.where(T['startsite']==s)[0]
        I = np.random.choice(I, int(len(I)/2))
        nvisit = len(np.where(T['visitedgalapagos'][I,0]==1)[0])
        pct_visitgalapagos[s] = 100.*nvisit/len(I)
        startlat[s] = T['lat'][I[0], 0]
    plt.plot(startlat, pct_visitgalapagos,'.-', color=exp_colors[exp], label=titles[exp])

plt.xlabel('Starting latitude [degrees N]')
plt.ylabel('Fraction of particles passing through Galapagos [%]')
plt.legend()
ylim = [0, 30]
xlim = [-25, 25]
for s in range(1, len(countrylats)):
    plt.plot([countrylats[s], countrylats[s]], ylim, 'k:')
    xmin = max(xlim[0], countrylats[s-1])
    xmax = min(xlim[1], countrylats[s])
    plt.text((xmin+xmax)/2, ylim[1]-8, countrynames[s-1], horizontalalignment='center', rotation=90)
plt.xlim(xlim)
plt.ylim(ylim)

plt.tight_layout()
savefig('lathistogram_Americas_halfdata.pdf')

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 7)
if 'bwd' in exp:
    projection = cartopy.crs.SouthPolarStereo(central_longitude=-90)
else:
    projection = cartopy.crs.PlateCarree(central_longitude=180)

for i, exp in enumerate(exps):
    T = D[exp]
    fig, ax = plt.subplots(1, 1,subplot_kw={'projection': projection})
    ax.coastlines()
    ax.add_feature(cartopy.feature.LAND)
    if 'bwd' in exp:
        gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=False,
                          linewidth=1, color='gray', alpha=0.5, linestyle='-')
        gl.xlines = False
        gl.ylocator = mticker.FixedLocator([-60, -30, 0, 30])


    ax.add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]],
                 width=galapagos_extent[1]-galapagos_extent[0], 
                 height=galapagos_extent[3]-galapagos_extent[2], facecolor='red', alpha=0.2,
                 transform=cartopy.crs.Geodetic()))

    plottimes = np.arange(np.min(T['time']), np.max(T['time']), dtype='datetime64[5D]')

    b = np.where(np.logical_and(T['time'] >= plottimes[0], T['time'] < plottimes[1]))
    scat = ax.scatter(T['lon'][b], T['lat'][b], s=3, c=T['visitedgalapagos'][b], 
                      transform=cartopy.crs.Geodetic(), cmap="bwr")
    if 'bwd' in exp:
        ax.set_xlim([-14678464.887996294, 8843973.281354474])
        ax.set_ylim([-12555957.896613397, 22597393.47540219])
    else:
        if 'americas'in exp:
            axextent = [-260, -60, -40, 35]
        elif 'fish'in exp:
            axextent = [-260, -60, -60, 45]            
        elif exp == 'china':
            axextent = [-90, 120, -20, 50] 
        ax.set_extent(axextent, crs=cartopy.crs.PlateCarree())
    scat.set_clim(0, 1)

    ttl = ax.set_title(titles[exp] + ' on ' + str(plottimes[0]) + ' (red particles pass through Galapagos)')
    frames = np.arange(0, len(plottimes)-1)

    def animate(t):
        b = np.where(np.logical_and(T['time'] >= plottimes[t], T['time'] < plottimes[t+1]))
        scat.set_offsets(np.vstack((T['lon'][b], T['lat'][b])).transpose())
        scat.set_array(T['visitedgalapagos'][b])
        ttl.set_text(titles[exp] + ' on ' + str(plottimes[t]) + ' (red particles pass through Galapagos)')
        return scat,

    if 'bwd' in exp:
        ax.set_xlim([-14678464.887996294, 8843973.281354474])
        ax.set_ylim([-12555957.896613397, 22597393.47540219])
    else:
        if 'americas'in exp:
            axextent = [-260, -60, -40, 35]
        elif 'fish'in exp:
            axextent = [-260, -60, -60, 45]            
        elif exp == 'china':
            axextent = [-90, 120, -20, 50] 
        ax.set_extent(axextent, crs=cartopy.crs.PlateCarree())
    anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=True)
    anim
    anim.save('galapagosparticles_%s.gif' % exp, writer='imagemagick', fps=10)

In [ ]:
exps = ['fishing', 'fishing_wstokes']
titles = {'fishing': 'currents from fisheries', 'fishing_wstokes': 'currents+waves from fisheries'}
exp_colors = {'fishing': 'C8', 'fishing_wstokes': 'C9'}

D = {}
for exp in exps:
    D[exp] = load_data(exp)

In [ ]:
from matplotlib import colors
pylab.rcParams['figure.figsize'] = (8, 10)

projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(len(D), 1, subplot_kw={'projection': projection})
if len(D) == 1:
    ax = [ax]

for i, exp in enumerate(exps):
    T = D[exp]
    ax[i].coastlines()
    ax[i].add_feature(cartopy.feature.LAND)

    bins = [np.arange(-260, -60, 1), np.arange(-60, 45, 1)]

    ax[i].add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                    height=galapagos_extent[3]-galapagos_extent[2], facecolor='none', alpha=1, linewidth=1,
                                    edgecolor='k', transform=cartopy.crs.Geodetic()))

    H, xe, ye = np.histogram2d(T['lon'][~np.isnan(T['lon'])], T['lat'][~np.isnan(T['lat'])], bins=bins, weights=T['fishinghours'][~np.isnan(T['lat'])])
    xb = (xe[1:] + xe[:-1])/2
    yb = (ye[1:] + ye[:-1])/2

    levels = [10**x for x in range(0, 9)]
    co = ax[i].contourf(xb, yb, H.T, levels=levels, transform=cartopy.crs.PlateCarree(), norm=colors.LogNorm())
    ax[i].set_extent([bins[0][0], bins[0][-1], bins[1][0], bins[1][-1]], crs=cartopy.crs.PlateCarree())
    ax[i].set_title(titles[exp])

cbaxes = fig.add_axes([0.1, 0.05, 0.8, 0.03]) 
cb = plt.colorbar(co, orientation="horizontal", cax = cbaxes)

# plt.tight_layout()
savefig('histogram_fishing.pdf')

In [ ]:
pylab.rcParams['figure.figsize'] = (8, 5)

import matplotlib.cm as cm


npzfile = np.load('fishingeffort.npz')
fishinghours_mat = npzfile['arr_0']
fishinghours_x = npzfile['arr_1'].T
fishinghours_y = npzfile['arr_2'].T
fishlon = []
fishlat = []
fishweight = []
for i in range(fishinghours_mat.shape[0]):
    for j in range(fishinghours_mat.shape[1]):
        if fishinghours_mat[i, j] > 24:
            fishlon.append(fishinghours_x[i, j])
            fishlat.append(fishinghours_y[i, j])
            fishweight.append(fishinghours_mat[i, j])

fishweight = np.array(fishweight)
fishlon = np.array(fishlon)
fishlat = np.array(fishlat)

lon_min, lon_max, lat_min, lat_max = [-102, -75, -15, 8]

indices = np.argsort(fishweight)

projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': projection})
ax.coastlines(resolution='50m')
ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.BORDERS)
gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
gl.xlocator = mticker.FixedLocator(np.arange(-105, -60, 5))
gl.ylocator = mticker.FixedLocator(np.arange(-15, 15, 2.5))

cmap = plt.get_cmap('inferno', 15)

sc = ax.scatter(fishlon[indices], fishlat[indices], c=fishweight[indices], transform=cartopy.crs.PlateCarree(), cmap=cmap, vmin=0, vmax=750)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=cartopy.crs.PlateCarree())
ax.add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                   height=galapagos_extent[3]-galapagos_extent[2], facecolor='red', alpha=0.25, linewidth=1,
                                   edgecolor='k', transform=cartopy.crs.Geodetic()))

ax.set_title('Total number of fishing hours in region around Galapagos')

plt.colorbar(sc)
plt.tight_layout()
savefig('fishinghoursmap.pdf')

In [ ]:
pylab.rcParams['figure.figsize'] = (12, 5)
projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(1, len(D), subplot_kw={'projection': projection})
if len(D) == 1:
    ax = [ax]

for i, exp in enumerate(exps):
    T = D[exp]
    totvisit = T['visitedgalapagos'][:,0] * T['fishinghours'][:, 0]
    
    pct_visitgalapagos = np.zeros(T['nsites'])
    startlat = np.zeros(T['nsites'])
    startlon = np.zeros(T['nsites'])
    for s in range(T['nsites']):
        I = np.where(T['startsite']==s)[0]
        nvisit = len(np.where(T['visitedgalapagos'][I,0]==1)[0])
        if len(I) > 0:
            pct_visitgalapagos[s] = 100.*nvisit/len(I)
            startlon[s] = T['lon'][I[0], 0]
            startlat[s] = T['lat'][I[0], 0]
    print('%s : %.2f pct reaches Galapagos (max %d pct)' %(titles[exp], np.nansum(totvisit)/np.nansum(T['fishinghours'][:, 0])*100., np.max(pct_visitgalapagos)))
    I = np.where(pct_visitgalapagos > 0)[0]
    indices = np.argsort(pct_visitgalapagos[I])
    J = np.where(pct_visitgalapagos == 0)[0]
    
    cmap = plt.get_cmap('jet', 8)
    ax[i].coastlines(resolution='50m')
    ax[i].add_feature(cartopy.feature.LAND)
    ax[i].add_feature(cartopy.feature.BORDERS)
    gl = ax[i].gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True,
                         linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
    gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
    gl.xlocator = mticker.FixedLocator(np.arange(-105, -60, 5))
    gl.ylocator = mticker.FixedLocator(np.arange(-15, 15, 2.5))
    ax[i].scatter(startlon[J], startlat[J], c='gray', transform=cartopy.crs.PlateCarree())
    sc = ax[i].scatter(startlon[I[indices]], startlat[I[indices]], c=pct_visitgalapagos[I[indices]], transform=cartopy.crs.PlateCarree(), cmap=cmap, vmin=0, vmax=40)
    ax[i].set_extent([lon_min, lon_max, lat_min, lat_max], crs=cartopy.crs.PlateCarree())
    ax[i].add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                       height=galapagos_extent[3]-galapagos_extent[2], facecolor='red', alpha=0.25, linewidth=1,
                                       edgecolor='k', transform=cartopy.crs.Geodetic()))

    ax[i].set_title('Percentage of particles reaching Galapagos region\n for %s' % titles[exp])
cbaxes = fig.add_axes([0.1, 0.05, 0.8, 0.03]) 
cb = plt.colorbar(sc, orientation="horizontal", cax = cbaxes)

# plt.tight_layout()
savefig('fishingfractions.pdf')